In [15]:
import sys
sys.path.append('/scratch/ty296/CT_MPS_mini')
from read_hdf5_func import von_neumann_entropy_sv

In [16]:
import h5py
import glob
import os
import tqdm
from typing import Dict, List, Tuple, Optional
import numpy as np
import pandas as pd

def postprocessing(sv_combined="/scratch/ty296/hdf5_data_combined/sv_combined.h5", dir_name='/scratch/ty296/hdf5_data/p_ctrl0.4'):
    h5_files = glob.glob(os.path.join(dir_name, '*.h5'))
    with h5py.File(sv_combined, 'w') as f_combined:
        counter = 0
        for file in tqdm.tqdm(h5_files):
            with h5py.File(file, 'r') as f:
                metadata = f['metadata']
                singular_values = f['singular_values']
                for result_group in metadata.keys():
                    counter += 1
                    p_proj = metadata[result_group]['p_proj'][()]
                    p_ctrl = metadata[result_group]['p_ctrl'][()]
                    L = metadata[result_group]['args']['L'][()]
                    maxbond = metadata[result_group]['max_bond'][()]
                    sv_arr = singular_values[result_group][()]
                    group_name = f'real{counter}'
                    grp = f_combined.create_dataset(group_name, data=sv_arr)
                    grp.attrs['p_proj'] = p_proj
                    grp.attrs['p_ctrl'] = p_ctrl
                    grp.attrs['L'] = L
                    grp.attrs['maxbond'] = maxbond

def calculate_mean_and_error(ee_values: List[float]) -> Tuple[float, float]:
    """Calculate mean and standard error of the mean."""
    ee_array = np.array(ee_values)
    mean = np.mean(ee_array)
    sem = np.std(ee_array, ddof=1) / np.sqrt(len(ee_array))
    return mean, sem

def calculate_variance_and_error(ee_values: List[float]) -> Tuple[float, float]:
    """Calculate variance and standard error of variance."""
    ee_array = np.array(ee_values)
    variance = np.var(ee_array, ddof=1)
    # Standard error of variance approximation
    n = len(ee_array)
    se_var = variance * np.sqrt(2.0 / (n - 1))
    return variance, se_var

def h5_to_csv(hdf5_combined, n, threshold, save_folder = '/scratch/ty296/plots'):
    with h5py.File(hdf5_combined, 'r') as f:
        from collections import defaultdict
        groups = defaultdict(list)
        for real_key in f.keys():
            s0 = von_neumann_entropy_sv(f[real_key][()], n=n, positivedefinite=False, threshold=threshold)
            # print(f[real_key].attrs['p_proj'],f[real_key].attrs['p_ctrl'],f[real_key].attrs['L'],f[real_key].attrs['maxbond'],s0)
            key_val = (f[real_key].attrs['L'],f[real_key].attrs['p_ctrl'],f[real_key].attrs['p_proj'])
            groups[key_val].append(s0)
        

        data = []
        for key_val, s0_list in groups.items():
            mean, sem = calculate_mean_and_error(s0_list)
            variance, se_var = calculate_variance_and_error(s0_list)
            # print(key_val, "mean", mean, "sem", sem, "variance", variance, "se_var", se_var)
            data.append(list(key_val) + [mean, sem, variance, se_var])
        print(data)

        df = pd.DataFrame(data, columns=['L', 'p_ctrl', 'p_proj', 'mean', 'sem', 'variance', 'se_var'])
        # save the data to a csv file
        csv_path = os.path.join(save_folder, f's{n}_threshold{threshold}.csv')
        df.to_csv(csv_path, index=False)

        return df

def plot_from_csv(csv_path):
    """
    Plot p_proj vs mean±sem and p_proj vs variance±se_var from CSV file
    CSV should have columns: L, p_ctrl, p_proj, mean, sem, variance, se_var
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Read CSV data
    df = pd.read_csv(csv_path)
    
    # Organize data by L values
    plot_data = {}
    for _, row in df.iterrows():
        L = row['L']
        if L not in plot_data:
            plot_data[L] = {'p_proj': [], 'mean': [], 'sem': [], 'variance': [], 'se_var': []}
        
        plot_data[L]['p_proj'].append(row['p_proj'])
        plot_data[L]['mean'].append(row['mean'])
        plot_data[L]['sem'].append(row['sem'])
        plot_data[L]['variance'].append(row['variance'])
        plot_data[L]['se_var'].append(row['se_var'])

    # Create plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Plot 1: p_proj vs mean ± sem
    for L in sorted(plot_data.keys()):
        data = plot_data[L]
        # Sort by p_proj for cleaner lines
        sorted_indices = np.argsort(data['p_proj'])
        p_proj_sorted = np.array(data['p_proj'])[sorted_indices]
        mean_sorted = np.array(data['mean'])[sorted_indices]
        sem_sorted = np.array(data['sem'])[sorted_indices]
        
        ax1.errorbar(p_proj_sorted, mean_sorted, yerr=sem_sorted, 
                    label=f'L={L}', marker='o', capsize=5, capthick=2)

    ax1.set_xlabel('p_proj')
    ax1.set_ylabel('Mean Entropy ± SEM')
    ax1.set_title('Mean Entropy vs p_proj for Different L')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: p_proj vs variance ± se_var
    for L in sorted(plot_data.keys()):
        data = plot_data[L]
        # Sort by p_proj for cleaner lines
        sorted_indices = np.argsort(data['p_proj'])
        p_proj_sorted = np.array(data['p_proj'])[sorted_indices]
        variance_sorted = np.array(data['variance'])[sorted_indices]
        se_var_sorted = np.array(data['se_var'])[sorted_indices]
        
        ax2.errorbar(p_proj_sorted, variance_sorted, yerr=se_var_sorted, 
                    label=f'L={L}', marker='s', capsize=5, capthick=2)

    ax2.set_xlabel('p_proj')
    ax2.set_ylabel('Variance ± SE')
    ax2.set_title('Variance vs p_proj for Different L')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


In [ ]:
# Example usage:
# First generate CSV using h5_to_csv function:

if __name__ == "__main__":
    sv_combined = "/scratch/ty296/hdf5_data_combined/sv_combined.h5"
    dir_name = '/scratch/ty296/hdf5_data/p_ctrl0.4'
    save_folder = '/scratch/ty296/plots'
    n = 0
    threshold = 1e-1
    postprocessing(sv_combined, dir_name)

    df = h5_to_csv(sv_combined, n=0, threshold=1e-1)

    # Then plot from the generated CSV:
    csv_path = os.path.join(save_folder, f's{n}_threshold{threshold}.csv')
    plot_from_csv(csv_path)


  1%|▏         | 9/682 [00:08<09:59,  1.12it/s]


KeyboardInterrupt: 